In [101]:
from spacy.displacy.render import EntityRenderer
from IPython.core.display import display, HTML
parsedPDF = parser.from_file("Visitor-Guide_2020-english-feb3.pdf")

In [102]:
import pandas as pd
data = pd.read_csv("data.csv")

In [103]:
data.fillna(-1)

,event_name,event_date,event_location,event_location_url,event_time,event_description
0,FebFest 2020 Winter Park in Boucher Park,01-Mar-20,Boucher Park,https://www.visitkingston.ca/venue/boucher-park/,8:00 AM,Bundle up the little kids and take them to Bou...
1,Free Outdoor Public Skating in Springer Market...,01-Mar-20,Market Square,https://www.visitkingston.ca/venue/market-square/,8:00 AM,Spring Market Square - come and enjoy a lovely...
2,The 2020 Tim Hortons Brier,01-Mar-20,Leon's Centre,https://www.visitkingston.ca/venue/leons-centre/,9:00 AM,"IT'S HERE!! The 2020 Tim Hortons Brier, presen..."
3,Memorial Centre Farmers’ Market,01-Mar-20,-1,-1,10:00 AM,The Memorial Centre Farmers' Market is open ye...
4,Introduction to Encaustic painting,01-Mar-20,Art Noise Studio,https://www.visitkingston.ca/venue/art-noise-s...,11:00 AM,Enjoy an afternoon of working with wax with Cl...
5,Indoor Public Skating at Kingston Memorial Centre,01-Mar-20,-1,-1,12:00 PM,Enjoy indoor ice skating with the whole family...
6,Beers & Bites Tour of Kingston,01-Mar-20,Stone City Ales,https://www.visitkingston.ca/venue/stone-city-...,3:00 PM,Spend an afternoon tasting the best of Kingsto...
7,Llama Llama – LIVE!,01-Mar-20,The Grand Theatre,https://www.visitkingston.ca/venue/the-grand-t...,-1,From the beloved book series by Anna Dewdney &...
8,"The Greg Runions Jazz Septet: Something gold, ...",01-Mar-20,Isabel Bader Centre for Performing Arts,https://www.visitkingston.ca/venue/isabel-bade...,7:30 PM,Greg Runions and his all-star Canadian septet ...
9,The Brier Patch Fan Zone presents 6ix Stix,01-Mar-20,The Brier Patch Fan Zone,https://www.visitkingston.ca/venue/the-brier-p...,10:00 PM,Rhythm Works 6ix Stix provides a fully interac...


In [106]:
data['event_description'].dtypes

dtype('O')

In [107]:
data.dtypes

event_name            object
event_date            object
event_location        object
event_location_url    object
event_time            object
event_description     object
dtype: object

In [108]:
def custom_render(doc, df, column, options={}, page=False, minify=False, idx=0):
    """Overload the spaCy built-in rendering to allow custom part-of-speech (POS) tags.
    
    Keyword arguments:
    doc -- a spaCy nlp doc object
    df -- a pandas dataframe object
    column -- the name of of a column of interest in the dataframe
    options -- various options to feed into the spaCy renderer, including colors
    page -- rendering markup as full HTML page (default False)
    minify -- for compact HTML (default False)
    idx -- index for specific query or doc in dataframe (default 0)
    
    """
    renderer, converter = EntityRenderer, parse_custom_ents
    renderer = renderer(options=options)
    parsed = [converter(doc, df=df, idx=idx, column=column)]
    html = renderer.render(parsed, page=page, minify=minify).strip()  
    return display(HTML(html))

def parse_custom_ents(doc, df, idx, column):
    """Parse custom entity types that aren't in the original spaCy module.
    
    Keyword arguments:
    doc -- a spaCy nlp doc object
    df -- a pandas dataframe object
    idx -- index for specific query or doc in dataframe
    column -- the name of of a column of interest in the dataframe
    
    """
    if column in df.columns:
        entities = df[column][idx]
        ents = [{'start': ent[1], 'end': ent[2], 'label': ent[3]} 
                for ent in entities]
    else:
        ents = [{'start': ent.start_char, 'end': ent.end_char, 'label': ent.label_}
            for ent in doc.ents]
    return {'text': doc.text, 'ents': ents, 'title': None}

def render_entities(idx, df, options={}, column='named_ents'):
    """A wrapper function to get text from a dataframe and render it visually in jupyter notebooks
    
    Keyword arguments:
    idx -- index for specific query or doc in dataframe (default 0)
    df -- a pandas dataframe object
    options -- various options to feed into the spaCy renderer, including colors
    column -- the name of of a column of interest in the dataframe (default 'named_ents')
    
    """
    text = df['text'][idx]
    custom_render(nlp(text), df=df, column=column, options=options, idx=idx)

In [109]:
# colors for additional part of speech tags we want to visualize
options = {
    'colors': {'COMPOUND': '#FE6BFE', 'PROPN': '#18CFE6', 'NOUN': '#18CFE6', 'NP': '#1EECA6', 'ENTITY': '#FF8800'}
}

In [110]:
pd.set_option('display.max_rows', 1000) # edit how jupyter will render our pandas dataframes
pd.options.mode.chained_assignment = None # prevent warning about working on a copy of a dataframe

In [111]:
nlp = spacy.load('en_core_web_sm')

In [134]:
file = 'data.csv'
df = pd.read_csv("data.csv")

mini_df = df[:200]
mini_df.index = pd.RangeIndex(len(mini_df.index))

# comment this out to run on full dataset
df = mini_df

In [135]:
df

,event_name,event_date,event_location,event_location_url,event_time,event_description
0,FebFest 2020 Winter Park in Boucher Park,01-Mar-20,Boucher Park,https://www.visitkingston.ca/venue/boucher-park/,8:00 AM,Bundle up the little kids and take them to Bou...
1,Free Outdoor Public Skating in Springer Market...,01-Mar-20,Market Square,https://www.visitkingston.ca/venue/market-square/,8:00 AM,Spring Market Square - come and enjoy a lovely...
2,The 2020 Tim Hortons Brier,01-Mar-20,Leon's Centre,https://www.visitkingston.ca/venue/leons-centre/,9:00 AM,"IT'S HERE!! The 2020 Tim Hortons Brier, presen..."
3,Memorial Centre Farmers’ Market,01-Mar-20,NaN,NaN,10:00 AM,The Memorial Centre Farmers' Market is open ye...
4,Introduction to Encaustic painting,01-Mar-20,Art Noise Studio,https://www.visitkingston.ca/venue/art-noise-s...,11:00 AM,Enjoy an afternoon of working with wax with Cl...
5,Indoor Public Skating at Kingston Memorial Centre,01-Mar-20,NaN,NaN,12:00 PM,Enjoy indoor ice skating with the whole family...
6,Beers & Bites Tour of Kingston,01-Mar-20,Stone City Ales,https://www.visitkingston.ca/venue/stone-city-...,3:00 PM,Spend an afternoon tasting the best of Kingsto...
7,Llama Llama – LIVE!,01-Mar-20,The Grand Theatre,https://www.visitkingston.ca/venue/the-grand-t...,NaN,From the beloved book series by Anna Dewdney &...
8,"The Greg Runions Jazz Septet: Something gold, ...",01-Mar-20,Isabel Bader Centre for Performing Arts,https://www.visitkingston.ca/venue/isabel-bade...,7:30 PM,Greg Runions and his all-star Canadian septet ...
9,The Brier Patch Fan Zone presents 6ix Stix,01-Mar-20,The Brier Patch Fan Zone,https://www.visitkingston.ca/venue/the-brier-p...,10:00 PM,Rhythm Works 6ix Stix provides a fully interac...


In [136]:
lower = lambda x: x.lower() # make everything lowercase

In [137]:
df = pd.DataFrame(df['event_description'].apply(lower))
df.columns = ['text']
display(df)

,text
0,bundle up the little kids and take them to bou...
1,spring market square - come and enjoy a lovely...
2,"it's here!! the 2020 tim hortons brier, presen..."
3,the memorial centre farmers' market is open ye...
4,enjoy an afternoon of working with wax with cl...
5,enjoy indoor ice skating with the whole family...
6,spend an afternoon tasting the best of kingsto...
7,from the beloved book series by anna dewdney &...
8,greg runions and his all-star canadian septet ...
9,rhythm works 6ix stix provides a fully interac...


In [138]:
def extract_named_ents(text):
    """Extract named entities, and beginning, middle and end idx using spaCy's out-of-the-box model. 
    
    Keyword arguments:
    text -- the actual text source from which to extract entities
    
    """
    return [(ent.text, ent.start_char, ent.end_char, ent.label_) for ent in nlp(text).ents]

def add_named_ents(df):
    """Create new column in data frame with named entity tuple extracted.
    
    Keyword arguments:
    df -- a dataframe object
    
    """
    df['named_ents'] = df['text'].apply(extract_named_ents)    

In [139]:
add_named_ents(df)
display(df)

,text,named_ents
0,bundle up the little kids and take them to bou...,"[(the whole month of february, 214, 241, DATE)]"
1,spring market square - come and enjoy a lovely...,"[(hours, 189, 194, TIME), (8 am to 10 pm, 217,..."
2,"it's here!! the 2020 tim hortons brier, presen...","[(2020, 16, 20, DATE), (8, 87, 88, TIME)]"
3,the memorial centre farmers' market is open ye...,"[(year-around, 44, 55, DATE)]"
4,enjoy an afternoon of working with wax with cl...,"[(afternoon, 9, 18, TIME)]"
5,enjoy indoor ice skating with the whole family...,"[(sunday, 80, 86, DATE), (the winter season, 9..."
6,spend an afternoon tasting the best of kingsto...,"[(an afternoon, 6, 18, TIME)]"
7,from the beloved book series by anna dewdney &...,[]
8,greg runions and his all-star canadian septet ...,"[(1996, 153, 157, DATE), (2017, 162, 166, DATE)]"
9,rhythm works 6ix stix provides a fully interac...,"[(6ix, 13, 16, ORDINAL), (40 minute, 102, 111,..."


In [143]:
column = 'named_ents'
render_entities(180, df, options=options, column=column) # take a look at one of the abstracts